In [1]:
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import MyVision
from MyVision.dataset.Dataset import DatasetUtils
from MyVision.engine.Engine import Trainer

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
    batch_size=128, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
    batch_size=128, shuffle=True
)

In [5]:
torchvision.models.resnet18(pretrained=True).conv1

Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

In [6]:
model = torchvision.models.resnet18(pretrained=True)
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(in_features=model.fc.in_features, out_features=10)
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(model.parameters(), lr=0.01)

In [7]:
trainer = Trainer(
    train_loader=train_loader,
    val_loader=test_loader,
    test_loader=None,
    device='cuda',
    loss=loss,
    optimizer=optimizer,
    model=model.to('cuda'),
    lr_scheduler=None
)

In [8]:
trainer.fit(10, 'accuracy')

  0%|                                                                                                                                                          | 0/469 [00:00<?, ?it/s]

[SAVING].....
  Epoch    Train loss    Validation loss    accuracy
-------  ------------  -----------------  ----------
      0      0.723506           0.201856      0.9411


  0%|                                                                                                                                                          | 0/469 [00:00<?, ?it/s]

[SAVING].....
  Epoch    Train loss    Validation loss    accuracy
-------  ------------  -----------------  ----------
      0      0.723506           0.201856      0.9411
      1      0.166893           0.122769      0.9642


  0%|                                                                                                                                                          | 0/469 [00:00<?, ?it/s]

...

[SAVING].....
  Epoch    Train loss    Validation loss    accuracy
-------  ------------  -----------------  ----------
      0     0.723506           0.201856       0.9411
      1     0.166893           0.122769       0.9642
      2     0.0971812          0.0962174      0.9705
      3     0.0670155          0.0857008      0.9736
      4     0.047651           0.080579       0.9764
      5     0.0347233          0.0805457      0.9773
      6     0.0276015          0.078512       0.9778
      7     0.0196391          0.0783193      0.979


  0%|                                                                                                                                                          | 0/469 [00:00<?, ?it/s]

[SAVING].....
  Epoch    Train loss    Validation loss    accuracy
-------  ------------  -----------------  ----------
      0     0.723506           0.201856       0.9411
      1     0.166893           0.122769       0.9642
      2     0.0971812          0.0962174      0.9705
      3     0.0670155          0.0857008      0.9736
      4     0.047651           0.080579       0.9764
      5     0.0347233          0.0805457      0.9773
      6     0.0276015          0.078512       0.9778
      7     0.0196391          0.0783193      0.979
      8     0.0155108          0.0753034      0.9803


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [00:03<00:00, 25.00it/s]


[SAVING].....
  Epoch    Train loss    Validation loss    accuracy
-------  ------------  -----------------  ----------
      0     0.723506           0.201856       0.9411
      1     0.166893           0.122769       0.9642
      2     0.0971812          0.0962174      0.9705
      3     0.0670155          0.0857008      0.9736
      4     0.047651           0.080579       0.9764
      5     0.0347233          0.0805457      0.9773
      6     0.0276015          0.078512       0.9778
      7     0.0196391          0.0783193      0.979
      8     0.0155108          0.0753034      0.9803
      9     0.0137818          0.0788613      0.9789
